In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_doc = requests.get(url).text
soup = BeautifulSoup(html_doc, 'html.parser')
html_table = soup.find('table')
table_rows = html_table.find_all('tr')

# Add the rows into a dataframe
l = []
for tr in table_rows:  
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
toronto = pd.DataFrame(l, columns=["PostalCode", "Borough","Neighborhood"])

# Remove unwanted rows and characters
toronto = toronto[toronto.Borough != 'Not assigned']
toronto.drop([0], axis=0, inplace=True)
toronto['Neighborhood'] = toronto['Neighborhood'].str.rstrip('\n')

# Group rows by postal code and aggregate the neighborhoods
toronto = toronto.groupby(['PostalCode', 'Borough']).agg([('neighborhood', ', '.join)])
toronto.reset_index(inplace=True)
toronto.columns = ['PostalCode', 'Borough', 'Neighborhood']

# Rename neighborhoods that are "not assigned" with the name of the borough
import numpy as np
toronto['Neighborhood'] = np.where(toronto['Neighborhood'] == 'Not assigned', toronto['Borough'], toronto['Neighborhood'])
toronto.loc[85]

# Have a look at the resulting dataframe
toronto.head()

ConnectionError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /wiki/List_of_postal_codes_of_Canada:_M (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x0000002CD2583518>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',))